In [1]:
#data stuff
import pandas as pd

#Modelling
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction import text

#NLP
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import gensim

#other
from tqdm import tqdm_notebook

C:\Users\steph\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


# Cleaning

In [2]:
fl511 = pd.read_csv('./Datasets/lastweek_y.csv')

In [77]:
fl511.drop(columns=['User', 'User_ID', 'Geo', 'HashTag'], inplace=True)

In [151]:
def remove_url(tweet):
    ind = tweet.find('fl511.com')
    tweet = tweet[:ind]
    return tweet
def remove_time(tweet):
    ind = tweet.rfind('at')
    tweet = tweet[:ind]
    return tweet
def remove_inc(tweet):
    ind = tweet.find(' on ')
    tweet = tweet[ind+4:]
    if ' on ' in tweet:
        ind = tweet.find(' on ')
        tweet = tweet[ind+4:]
    return tweet
def remove_tail(tweet):
    ind = tweet.rfind(',')
    tweet = tweet[:ind]
    return tweet

In [152]:
fl511 = pd.read_csv('./Datasets/lastweek_y.csv')
fl511['Tweet'] = fl511['Tweet'].map(remove_url)
fl511['Tweet'] = fl511['Tweet'].map(remove_time)
fl511['Tweet'] = fl511['Tweet'].map(remove_inc)
fl511['Tweet'] = fl511['Tweet'].map(remove_tail)

In [172]:
roads = set()
for tweet in fl511['Tweet']:
    join_words = [' to ', ' before ', ' beyond ', ' at ', ' from ']
    easy = True
    if len(tweet) == 0:
        easy=False
    for stop in join_words:
        if stop in tweet:
            easy=False
    if  easy == True:
        roads.add(tweet)
    for join in join_words:
        ind = tweet.find(join)
        if ind != -1:
            loc1 = tweet[:ind]
            if 'from' not in loc1:
                roads.add(loc1)
            loc2 = tweet[ind + len(join):]
            if 'Exit' not in loc2:
                roads.add(loc2)
        #roads.add(loc2)
roads

{'826 Express south',
 '826 Express south ramp',
 '95 Express north',
 '95 Express north ramp',
 '95 Express south',
 '95 Express south ramp',
 'Alton Rd',
 'Bird Rd',
 'Bridge Rd/Star Island',
 'Caribbean Blvd',
 'Florida Turnpike',
 "Florida's Turnpike/SR-826",
 'Floridas Turnpike north',
 'Floridas Turnpike north ramp',
 'Floridas Turnpike south',
 'Floridas Turnpike south ramp',
 'Fountain St/Palm Island',
 'Golden Glades Park & Ride',
 'I-195 East - Miami Beach/SR-112 West - Airport',
 'I-195 east',
 'I-195 east ramp',
 'I-195 west',
 'I-195 west ramp',
 'I-395 east',
 'I-395 east ramp',
 'I-395 west',
 'I-75',
 'I-75 north',
 'I-75 south',
 'I-75 south ramp',
 'I-75 to',
 'I-95',
 'I-95 Mainline/Golden Glades Interchange',
 'I-95 Mainline/I-395',
 'I-95 Mainline/Ives Dairy Rd',
 'I-95 North',
 'I-95 South',
 'I-95 north',
 'I-95 north ramp',
 'I-95 south',
 'I-95 south ramp',
 'Le Jeune Rd',
 'Le Jeune Rd North',
 'Le Jeune Rd North and traffic backed up until NW 57 Ave',
 'Lejeu

In [173]:
tweets_20 = pd.read_csv('./Datasets/lastweek_X_20000.csv')

In [3]:
fl511 = pd.read_csv('./Datasets/lastweek_y.csv')

In [72]:
expr = '[A-Z]+\-\d+|[a-zA-z]+|[A-Z][a-zA-Z]+\s\d{1,3}[A-Z]*|\d{1,3}\W[A-Z][a-zA-Z]+'

In [176]:
test = 'i-85'

card = re.compile(r'(?:\d{1,4}|[A-Z][a-z]+\W|[A-Za-z]+\-\d+){1,3}')
re.findall(card, test)

['i-85']

# Format X

In [4]:
data = pd.read_csv('Datasets/timeloop_2019-07-23.csv')
data.drop(columns=['User','User_ID','Geo'], inplace = True)
data['Date'] = pd.to_datetime(data['Date'])
data['Date'] = data.assign(Date=data['Date'].dt.round('H'))['Date']
times = []
times.append(data['Date'][0])

for i in tqdm_notebook(data.index):
    time = data['Date'][i]
    if time != times[len(times)-1]:
        times.append(time)
        
dic = {'time': [], 'tweets': []}

for hour in tqdm_notebook(times):
    total = ''
    tweets = list(data[data['Date'] == hour]['Tweet'])
    for twit in tweets:
        total += str(twit)
    dic['time'].append(hour)
    dic['tweets'].append(total)
data2 = pd.DataFrame.from_dict(dic)

data2.set_index('time', inplace=True)
data2.sort_index(inplace=True)

In [7]:
data2.head()

,tweets
time,
2019-07-22 11:00:00+00:00,Lmaoo wah kinda stress thisOjala algun puto di...
2019-07-22 12:00:00+00:00,REAR VIEW (Intro) - Hell Storm @Greater Downto...
2019-07-22 13:00:00+00:00,Welp. pic.twitter.com/kiRmEMKoGTYou are beginn...
2019-07-22 14:00:00+00:00,EFE: Pedro Sánchez aborda plan de gobierno en ...
2019-07-22 15:00:00+00:00,About yesterday. 1st to Brunch at #latinhouseg...


# Format Y

In [10]:
closed = pd.read_csv('Datasets/manual_y.csv')
closed.rename({'Unnamed: 0': 'Date'}, axis=1, inplace=True)
closed['Date'] = pd.to_datetime(closed['Date'])
closed.set_index('Date', inplace=True)
closed.sort_index(inplace=True)

# Word Vec (TODO)

In [6]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/steph/Downloads/lexvec.enwiki+newscrawl.300d.W.pos.vectors/lexvec.enwiki+newscrawl.300d.W.pos.vectors')

In [ ]:
# from gensim.models.word2vec import Word2Vec

# #replace with own corpus later
# import gensim.downloader as api
# corpus = api.load('text8')

# # Train a model! 
# model = Word2Vec(corpus,      # Corpus of data.
#                  size=100,    # How many dimensions do you want in your word vector?
#                  window=5,    # How many "context words" do you want?
#                  min_count=1, # Ignores words below this threshold.
#                  sg=0,        # SG = 1 uses SkipGram, SG = 0 uses CBOW (default).
#                  workers=2)   # Number of "worker threads" to use (parallelizes process).

[==================================================] 100.0% 31.6/31.6MB downloaded


C:\Users\steph\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


# Split

In [9]:
#only for one road at the moment
X = data2
y = closed[''] #column
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
#framework for multiple models, untested
for road_i in range(closed.shape[1]):
    exec(f'X_train_{i}, X_test_{i}, y_train_{i}, y_test_{i}, train_test_split(X, y={closed[closed.columns[i]]}, stratify={closed[closed.columns[i]]}, random_state=42)')

# Modeling

In [ ]:
pipe = Pipeline([
    ('vec', TfidfVectorizer(token_pattern='[a-zA-z]+ | [A-Za-z]+\-*\d+\W(?:[sS]outh|[Nn]orth|East|West|[NSEW]{1,2}|[nswe]{1,2})*')),
    ('nb', MultinomialNB())
])
pipe_params = {
    'vec__stop_words': [custom, 'english'],
    'vec__max_features': [12000],
    'vec__min_df': [1, 2],
    'vec__max_df': [.9],
    'vec__ngram_range': [(3,5),(3,7)],
    'nb__alpha': [1]
}

In [23]:
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print('cvs:', gs.best_score_)
print('train score:', gs.score(X_train, y_train))
print('test score:', gs.score(X_test, y_test))
gs.best_params_b

NameError: name 'pipe' is not defined

In [ ]:
#framework for multiple models
for road_i in range(closed.shape[1]):
    exec(f'gs_{i} = GridSearchCV(pipe, param_grid=pipe_params, cv=5)')
    exec(f'gs_{i}.fit(X_train_{i}, y_train_{i})')
    
score = 0
for road_i in range(closed.shape[1]):
    score += exec(f'gs_{i}.score(X_test_{i}, y_test_{i})')
score = score / closed.shape[1]
score